In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.layers import concatenate, Dropout, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255

y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

In [4]:
x_letter = train['letter'].values
x_letter = x_letter[:, np.newaxis]
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, shuffle=True)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

x_letter_train = x_letter[:x_train.shape[0],:]
x_letter_val = x_letter[x_train.shape[0]:,:]
print(x_letter_train.shape)
print(x_letter_val.shape)

(1843, 28, 28, 1)
(205, 28, 28, 1)
(1843, 10)
(205, 10)
(1843, 26)
(205, 26)


In [6]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (7,7), padding='same', strides=2,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((3,3), padding='same', strides=2)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    
    return model

In [7]:
def DenseNet_letter(model_input, letter, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (7,7), padding='same', strides=2,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((3,3), padding='same', strides=2)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    merge = concatenate([x, letter])
    x1 = Dense(500, activation='relu')(merge)
    x1 = Dropout(0.3)(x1)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x1)
    
    model = Model(inputs = [model_input, letter], outputs = model_output, name=densenet_type)
    
    return model

In [8]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48]}

base_growth_rate = 32

model_input = Input(shape=(28,28,1))
letter = Input(shape=(26,))
classes = 10

model = DenseNet_letter(model_input, letter, classes, 'DenseNet-121')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 14, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 14, 14, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 14, 14, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [10]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy']) 

cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x_train, x_letter_train], y_train, validation_data=([x_val, x_letter_val], y_val),
                    batch_size=64, epochs=100, callbacks = [cp])

Train on 1843 samples, validate on 205 samples
Epoch 1/100
1843/1843 [==============================] - 37s 20ms/step - loss: 2.1712 - acc: 0.2355 - val_loss: 2.2457 - val_acc: 0.1512
Epoch 2/100
1843/1843 [==============================] - 9s 5ms/step - loss: 1.6733 - acc: 0.4107 - val_loss: 2.3092 - val_acc: 0.1463
Epoch 3/100
1843/1843 [==============================] - 9s 5ms/step - loss: 1.3539 - acc: 0.5263 - val_loss: 2.8185 - val_acc: 0.0927
Epoch 4/100
1843/1843 [==============================] - 9s 5ms/step - loss: 1.0552 - acc: 0.6457 - val_loss: 3.2731 - val_acc: 0.0927
Epoch 5/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.7745 - acc: 0.7379 - val_loss: 6.2198 - val_acc: 0.0927
Epoch 6/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.6784 - acc: 0.7710 - val_loss: 4.5338 - val_acc: 0.0829
Epoch 7/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.5773 - acc: 0.7987 - val_loss: 12.2883 - val_acc: 0.0927
Epoch

Epoch 61/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0788 - acc: 0.9816 - val_loss: 2.9230 - val_acc: 0.6049
Epoch 62/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0682 - acc: 0.9799 - val_loss: 2.3595 - val_acc: 0.6146
Epoch 63/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0716 - acc: 0.9821 - val_loss: 2.1104 - val_acc: 0.6293
Epoch 64/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0413 - acc: 0.9870 - val_loss: 3.4737 - val_acc: 0.5463
Epoch 65/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0425 - acc: 0.9875 - val_loss: 2.2766 - val_acc: 0.6195
Epoch 66/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0476 - acc: 0.9881 - val_loss: 3.2933 - val_acc: 0.5659
Epoch 67/100
1843/1843 [==============================] - 9s 5ms/step - loss: 0.0334 - acc: 0.9891 - val_loss: 3.1534 - val_acc: 0.5707
Epoch 68/100
1843/1843 [========================